In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
'''trainEvents is the list of all sound events across all training clips.
evalEvents is the list of all sound events across all eval clips.
displayNames is a table for Google's labeling scheme.
See https://research.google.com/audioset/download.html for more details.
'''

trainEvents = pd.read_csv("../data/audioset_train_strong.tsv", sep="\t")
evalEvents = pd.read_csv("../data/audioset_eval_strong_framed_posneg.tsv", sep="\t")
displayNames = pd.read_csv("../data/mid_to_display_name.tsv", sep="\t",header=None)

trainEvents.head()

,segment_id,start_time_seconds,end_time_seconds,label
0,b0RFKhbpFJA_30000,0.000,10.000,/m/03m9d0z
1,b0RFKhbpFJA_30000,4.753,5.720,/m/05zppz
2,b0RFKhbpFJA_30000,0.000,10.000,/m/07pjwq1
3,b0RFKhbpFJA_30000,6.899,7.010,/m/07qjznt
4,b0RFKhbpFJA_30000,8.534,9.156,/t/dd00092


In [4]:
# For EDA on this part, need to see how well represented each label is.
# Make a bar graph where each bar height = # times a certain event is found.
# Make another bar graph where each bar height = total # seconds a certain event is found.


In [5]:
evalEvents.head()

,segment_id,start_time_seconds,end_time_seconds,label,present
0,YxlGt805lTA_30000,0.0,0.96,/m/02zsn,NOT_PRESENT
1,YxlGt805lTA_30000,0.0,0.96,/m/04qvtq,NOT_PRESENT
2,YxlGt805lTA_30000,0.0,0.96,/m/04rlf,PRESENT
3,YxlGt805lTA_30000,0.0,0.96,/m/07rgkc5,NOT_PRESENT
4,YxlGt805lTA_30000,0.0,0.96,/m/0jbk,NOT_PRESENT


In [6]:
displayNames.head()

,0,1
0,/g/11b630rrvh,Kettle whistle
1,/g/122z_qxw,Firecracker
2,/m/01280g,Wild animals
3,/m/012f08,Motor vehicle (road)
4,/m/012n7d,Ambulance (siren)


In [7]:
import tensorflow as tf

In [8]:
from os import listdir
from os.path import isfile, join
def get_filenames(mypath):
    return [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [9]:
'''
The dataset goes frame by frame, with a spectral analysis of each one's
(very short) snippet of sound.

The feature sets are individual .tfrecord files separated into folders.
Read THIS for a tutorial on building a pipeline to deal with them.
https://www.tensorflow.org/guide/data#consuming_tfrecord_data

Still trying to figure out how to wrangle this.
----------------------------------------------------------------------------------

This has been wrangled, I think. With the function defined above and the 
changes below, we can read our entire dataset into a TFRecordDataset.
Next step is to figure out how to make sense of those 8-bit audio features.
'''
#filenames = ["../data/audioset_v1_embeddings/bal_train/-n.tfrecord","../data/audioset_v1_embeddings/bal_train/--.tfrecord"]
filenames = ["../data/audioset_v1_embeddings/bal_train/" + i for i in get_filenames("../data/audioset_v1_embeddings/bal_train/")]
#print(filenames)
raw_dataset = tf.data.TFRecordDataset(filenames)

for raw_record in raw_dataset.take(3):
    example = tf.train.SequenceExample()
    example.ParseFromString(raw_record.numpy())
    print(example)

context {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 399
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 30.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "--cB2ZVjpnA"
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "audio_embedding"
    value {
      feature {
        bytes_list {
          value: "_z\223\016e0\235}\372\365j\2513~M\221\205%\265\237 \373\327\210\240\367\232+_\033\210`q=H\377J[\247j\003\230p\377\371a\334\3770\377\177\377*\030\211\251\226\340\000C\370\246:T\000\2177d\253\377|E\177\006\000\000+\000\377\207EO\323\261\321\272\342Mk\225\033\377\036\377\000\034\207\377\374o_\377\233\\C\343Na\000\377\000\000\333q\377\232m\177\271\000\272\000?\377\377\002\276\000"
        }
      }
      feature {
        bytes_list 